<a href="https://colab.research.google.com/github/emredeveloper/Selenium-Projects/blob/main/Selenium_%2B_Embed_%2B_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CLAUDE 3.5 SONNET ile yapılmış sorgu verilmiştir

# Wikipedia Veri Bilimi Sayfası Analizi

Bu script aşağıdaki işlemleri gerçekleştirir:

1. **Argüman Ayrıştırma**
   - Kullanıcıdan başlık sayısı, dil ve konu bilgilerini alır.

2. **Wikipedia'dan Veri Çekme**
   - Selenium kullanarak belirtilen Wikipedia sayfasını açar.
   - İstenilen sayıda başlık ve ilk cümleleri çeker.

3. **Veri Ön İşleme**
   - Çekilen metinleri küçük harfe çevirir.
   - Noktalama işaretlerini kaldırır.
   - Stop words'leri temizler.

4. **Embedding Oluşturma**
   - Sentence Transformers kullanarak metinleri vektörlere dönüştürür.

5. **Benzerlik Analizi**
   - Cosine similarity kullanarak cümleler arası benzerliği hesaplar.
   - Benzerlik matrisini görselleştirir.
   - En benzer iki cümleyi bulur.

6. **Kelime Bulutu Oluşturma**
   - Tüm metinlerden bir kelime bulutu oluşturur ve görselleştirir.

7. **Kelime Frekans Analizi**
   - En sık kullanılan 10 kelimeyi ve frekanslarını hesaplar.

8. **Duygu Analizi**
   - TextBlob kullanarak her cümlenin duygu polaritesini hesaplar.
   - Ortalama duygu polaritesini hesaplar.

9. **Kümeleme Analizi**
   - K-means algoritması kullanarak cümleleri kümeler.
   - Kümeleme sonuçlarını yazdırır.

10. **Ortalama Benzerlik Hesaplama**
    - Her cümlenin diğer cümlelerle ortalama benzerliğini hesaplar.

11. **Sonuçları Kaydetme**
    - Tüm analiz sonuçlarını bir CSV dosyasına kaydeder.

12. **Ek Görselleştirmeler**
    - Kümelenmiş embedding'leri görselleştirir.
    - Duygu analizi sonuçlarını görselleştirir.

13. **Loglama**
    - Tüm işlem adımlarını loglar.

In [ ]:
%%writefile app.py
import argparse
import csv
import string
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from wordcloud import WordCloud
from textblob import TextBlob

# Argüman ayrıştırıcı ekle
parser = argparse.ArgumentParser(description='Wikipedia Veri Bilimi sayfası analizi')
parser.add_argument('--num_topics', type=int, default=5, help='Analiz edilecek başlık sayısı')
parser.add_argument('--language', type=str, default='en', help='Wikipedia dili (örn. "en" veya "tr")')
parser.add_argument('--topic', type=str, default='Data_science', help='Wikipedia konusu (örn. "Data_science" veya "Veri_bilimi")')
args = parser.parse_args()

# Loglama ayarla
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# NLTK için gerekli verileri indir
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Metin ön işleme fonksiyonu
def preprocess_text(text, lang='english'):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    stop_words = set(stopwords.words(lang))
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if w not in stop_words]
    return ' '.join(filtered_text)

# Chrome options ayarla
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--headless")

# WebDriver'ı başlat
driver = webdriver.Chrome(options=chrome_options)

# Wikipedia URL'sini dile ve konuya göre ayarla
wiki_url = f"https://{args.language}.wikipedia.org/wiki/{args.topic}"
driver.get(wiki_url)

logging.info(f"Wikipedia sayfası açıldı: {wiki_url}")

# İlk n başlığı ve ilk cümleleri saklamak için bir liste
topics = []

# Başlıkları bul
headers = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h2 .mw-headline"))
)

# İlk n başlık için döngü (n, komut satırı argümanından gelir)
for i, header in enumerate(headers[:args.num_topics], 1):
    title = header.text
    # Başlığın altındaki ilk paragrafı bul
    try:
        paragraph = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, f"//h2[span[@class='mw-headline' and text()='{title}']]//following-sibling::p[1]"))
        )
        first_sentence = paragraph.text.split('.')[0] + '.'  # İlk cümleyi al
        topics.append([title, first_sentence])
        logging.info(f"Başlık {i} çekildi: {title}")
    except:
        logging.warning(f"Başlık {i} için paragraf bulunamadı: {title}")

# Tarayıcıyı kapat
driver.quit()

# CSV dosyasına yaz
with open('data_science_topics.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "First Sentence"])
    writer.writerows(topics)

logging.info("Veriler CSV dosyasına kaydedildi.")

# Cümleleri ayır
sentences = [topic[1] for topic in topics]

# Metinleri ön işle
preprocessed_sentences = [preprocess_text(sentence, 'english' if args.language == 'en' else 'turkish') for sentence in tqdm(sentences, desc="Preprocessing")]

# Embedding modeli yükle
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Ön işlenmiş metinleri embed et
embeddings = model.encode(preprocessed_sentences, show_progress_bar=True)

# Benzerlik matrisini hesapla
similarity_matrix = cosine_similarity(embeddings)

# Benzerlik matrisini görselleştir
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, annot=True, cmap='coolwarm')
plt.title('Similarity Matrix Heatmap')
plt.savefig('similarity_heatmap.png')
plt.close()

logging.info("Benzerlik matrisi oluşturuldu ve kaydedildi.")

# En benzer iki cümleyi bul
max_similarity = np.max(np.triu(similarity_matrix, k=1))
most_similar_pair = np.unravel_index(np.argmax(np.triu(similarity_matrix, k=1)), similarity_matrix.shape)

print(f"\nEn benzer iki cümle:")
print(f"1. {sentences[most_similar_pair[0]]}")
print(f"2. {sentences[most_similar_pair[1]]}")
print(f"Benzerlik: {max_similarity:.2f}")

# Kelime bulutu oluştur
all_text = ' '.join(preprocessed_sentences)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Data Science Topics')
plt.savefig('wordcloud.png')
plt.close()

logging.info("Kelime bulutu oluşturuldu ve kaydedildi.")

# En sık kullanılan kelimeleri bul
words = all_text.split()
word_freq = Counter(words)
print("\nEn sık kullanılan 10 kelime:")
for word, count in word_freq.most_common(10):
    print(f"{word}: {count}")

# Duygu analizi
sentiments = [TextBlob(sentence).sentiment.polarity for sentence in sentences]
avg_sentiment = sum(sentiments) / len(sentiments)
print(f"\nOrtalama duygu polaritesi: {avg_sentiment:.2f}")

# K-means kümeleme
num_clusters = min(3, len(sentences))  # En fazla 3 küme veya cümle sayısı kadar
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(embeddings)

# Kümeleme sonuçlarını yazdır
for i in range(num_clusters):
    print(f"\nKüme {i+1}:")
    cluster_sentences = [sentences[j] for j in range(len(sentences)) if cluster_labels[j] == i]
    for sentence in cluster_sentences:
        print(f"- {sentence}")

# Daha fazla analiz: Her cümlenin ortalama benzerliği
avg_similarities = np.mean(similarity_matrix, axis=1)
for i, avg_sim in enumerate(avg_similarities):
    print(f"Cümle {i+1} ortalama benzerlik: {avg_sim:.2f}")

# Sonuçları DataFrame'e dönüştür ve CSV'ye kaydet
results_df = pd.DataFrame({
    'Title': [topic[0] for topic in topics],
    'Sentence': sentences,
    'Preprocessed_Sentence': preprocessed_sentences,
    'Average_Similarity': avg_similarities,
    'Cluster': cluster_labels,
    'Sentiment': sentiments
})
# Embed değerlerini ekle
for i in range(embeddings.shape[1]):
    results_df[f'Embed_{i+1}'] = embeddings[:, i]
results_df.to_csv('data_science_analysis.csv', index=False)

logging.info("Sonuçlar 'data_science_analysis.csv' dosyasına kaydedildi.")

# Ek görselleştirmeler
plt.figure(figsize=(10, 6))
sns.scatterplot(x=embeddings[:, 0], y=embeddings[:, 1], hue=cluster_labels, palette='deep')
plt.title('Sentence Embeddings Clustered')
plt.savefig('embedding_clusters.png')
plt.close()

plt.figure(figsize=(10, 6))
sns.barplot(x=list(range(len(sentences))), y=sentiments)
plt.title('Sentiment Analysis of Sentences')
plt.xlabel('Sentence Index')
plt.ylabel('Sentiment Polarity')
plt.savefig('sentiment_analysis.png')
plt.close()

logging.info("Analiz tamamlandı. Tüm sonuçlar ve görseller kaydedildi.")

Overwriting app.py


In [ ]:
!python app.py --num_topics 7 --language tr --topic Veri_bilimi